In [1]:
from symtable import Symbol
from StockStageEstimator import StockStageEstimator
import pandas as pd
import numpy as np
import random
from datetime import datetime
from stage import fullPrint
# import mariadb
import sys
import os
# from dotenv import load_dotenv

# load_dotenv()

### ENV CONSTANT
DB_PASSWORD = os.getenv('DB_PASSWORD')
DB_HOST = os.getenv('DB_HOST')
DB_USER = os.getenv('DB_USER')

### Connect To DB
# try:
#     conn = mariadb.connect(
#         user=DB_USER,
#         password=DB_PASSWORD,
#         host=DB_HOST,
#         port=3306,
#         database="stock",
#         autocommit=True
#     )
# except mariadb.Error as e:
#     print(f"Error connecting to MariaDB Platform: {e}")
#     sys.exit(1)
# cur = conn.cursor()

### CONSTANT VARIABLES ###

## Mutation Noise 
mutationNoises = []
mutationNoises.append(np.random.normal(0, 0.025)) # 0 - Five Year High
mutationNoises.append(np.random.normal(0, 0.075)) # 1 - Volume Perc
mutationNoises.append(np.random.normal(0, 0.015)) # 2 - RS
mutationNoises.append(np.random.normal(0, 0.005)) # 3 - Slope
mutationNoises.append(np.random.normal(0, 0.0125)) # 4 - Price v WMA30
mutationNoises.append(np.random.normal(0, 0.015)) # 5 - Price v Initial Support
mutationNoises.append(np.random.normal(0, 0.005)) # 6 - Prev Trough v Peak
mutationNoises.append(np.random.normal(0, 0.01)) # 7 - Price v Prev Support
mutationNoises.append(np.random.normal(0, 0.0125)) # 8 - Price v Prev Close
mutationNoises.append(np.random.normal(0, 0.0005)) # 9 - Sector MA30 Slope
mutationNoises.append(np.random.normal(0, 0.0025)) # 10 - Sector RS
mutationNoises.append(np.random.normal(0, 0.0025)) # 11 - SPDF Price v WMA30
mutationNoises.append(np.random.normal(0, 0.0002)) # 12 - SPDF WMA30 Slope
mutationNoises.append(np.random.normal(0, 0.0025)) # 13 - SPDF Sell P v WMA30
mutationNoises.append(np.random.normal(0, 0.0002)) # 14 - SPDF Sell WMA30 Slope


In [23]:
## Get returns of a ticker list
def calculateGroupReturn(list):
    monthlyReturn = pd.read_pickle("transactionTemplate.pkl")
    monthlyReturn = monthlyReturn.resample('M').mean()
    monthlyReturn.index = monthlyReturn.index.strftime("%Y:%m")
    currentColumn = -1
    for industry in list:
        print(industry)
        symbol = industry[1]
        if symbol in monthlyReturn.columns:
            continue
        try:
            df = pd.read_pickle("stockData/industriesData/"+industry[0]+'/'+symbol+".pkl")
        except:
            print("error "+ symbol)
            continue
        if not (df['Close'] != 0).all() or df.empty:
            if df.empty:
                print("error "+ symbol)
            continue
        monthlyReturn[symbol] = 1
        currentColumn += 1
        monthCheck = monthlyReturn.index[0]
        lastPrice = df.iat[0,0]
        initialPrice = df.iat[0,0]
        for index, element in df.iterrows():
            if (index.strftime("%Y:%m") != monthCheck):
                lastPrice = element[0]
                if (lastPrice / initialPrice >= 1.01 and lastPrice / initialPrice < 2):
                    rowNumber = monthlyReturn.index.get_loc(index.strftime("%Y:%m"))
                    monthlyReturn.iat[rowNumber, currentColumn] = (lastPrice / initialPrice)
                initialPrice = lastPrice
                monthCheck = index.strftime("%Y:%m")
    return monthlyReturn

In [31]:
industryList = pd.read_pickle("stockData/industryList.pkl")
calculateGroupReturn(industryList)

('XLP', 'DJTobacco')
('XLP', 'DJSoftDrinks')
('XLP', 'DJDrugRetailers')
('XLP', 'DJDistillers&Vintners')
('XLP', 'DJFoodProductMakers')
('XLP', 'DJFoodRetail&Wholesale')
('XLP', 'DJNondurableHousehold')
('XLP', 'DJBrewers')
('XLP', 'DJPersonalProducts')
('XLU', 'DJGasDistribution')
('XLU', 'DJMultiutilities')
('XLU', 'DJConventionalElectricity')
('XLU', 'DJWater')
('XLV', 'DJPharmaceuticals')
('XLV', 'DJMedicalSupplies')
('XLV', 'DJHealthCareProviders')
('XLV', 'DJMedicalEquipment')
('XLV', 'DJBiotechnology')
('XLF', 'DJFinancialAdministration')
('XLF', 'DJInsuranceBrokers')
('XLF', 'DJSpecialtyFinance')
('XLF', 'DJFullLineInsurance')
('XLF', 'DJConsumerFinance')
('XLF', 'DJAssetManagers')
('XLF', 'DJInvestmentServices')
('XLF', 'DJReinsurance')
('XLF', 'DJProperty&CasualtyInsurance')
('XLF', 'DJLifeInsurance')
('XLF', 'DJMortgageFinance')
('XLF', 'DJBanks')
('XLE', 'DJCoal')
('XLE', 'DJIntegratedOil&Gas')
('XLE', 'DJPipelines')
('XLE', 'DJExploration&Production')
('XLE', 'DJOilEquipme

/var/folders/1q/w3g5nkks29g3kjdwmq1lssj00000gn/T/ipykernel_12554/3290754152.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  monthlyReturn[symbol] = 1
/var/folders/1q/w3g5nkks29g3kjdwmq1lssj00000gn/T/ipykernel_12554/3290754152.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  monthlyReturn[symbol] = 1
/var/folders/1q/w3g5nkks29g3kjdwmq1lssj00000gn/T/ipykernel_12554/3290754152.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performa

,DJTobacco,DJSoftDrinks,DJDrugRetailers,DJDistillers&Vintners,DJFoodProductMakers,DJFoodRetail&Wholesale,DJNondurableHousehold,DJBrewers,DJPersonalProducts,DJGasDistribution,...,DJRestaurants&Bars,DJHomeImprovementRetailers,DJFurnishings,DJSpecializedConsumerServices,DJHotels,DJRecreationalProducts,DJGambling,DJDurableHouseholdProducts,DJToys,DJApparelRetailers
Date,,,,,,,,,,,,,,,,,,,,,
2010:01,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
2010:02,1.000000,1.000000,1.000000,1.000000,1.000000,1.033395,1.015557,1.000000,1.000000,1.000000,...,1.022862,1.000000,1.000000,1.000000,1.000000,1.041203,1.000000,1.000000,1.000000,1.000000
2010:03,1.019468,1.043271,1.037862,1.090635,1.048254,1.075301,1.029537,1.109250,1.054945,1.070267,...,1.059317,1.121595,1.151375,1.109954,1.105065,1.111965,1.065971,1.084726,1.072679,1.140106
2010:04,1.044064,1.021579,1.075190,1.054702,1.012351,1.035329,1.000000,1.000000,1.020916,1.033605,...,1.053966,1.025789,1.014658,1.026069,1.138311,1.079582,1.195675,1.047131,1.052435,1.071904
2010:05,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021:08,1.000000,1.036429,1.000000,1.000000,1.000000,1.051083,1.019248,1.000000,1.000000,1.014698,...,1.034991,1.010490,1.000000,1.019052,1.000000,1.020838,1.000000,1.000000,1.000000,1.013560
2021:09,1.052869,1.000000,1.038200,1.000000,1.000000,1.048461,1.018422,1.000000,1.000000,1.000000,...,1.000000,1.026435,1.000000,1.000000,1.000000,1.000000,1.086581,1.000000,1.000000,1.000000
2021:10,1.000000,1.000000,1.000000,1.000000,1.012131,1.000000,1.000000,1.013196,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.134887,1.000000,1.000000,1.000000,1.000000,1.000000


In [ ]:
## Calculate Goodsector & Initialize Estimator Object 
def evalReturn(individual):
    now = datetime.now()
    now = now.strftime("%Y-%m-%d-%H:%M")
    transaction = pd.read_pickle("transactionTemplate.pkl")
    transaction['holding'] = np.empty((len(transaction), 0)).tolist()
    SectorDict = {
    "Energy":"XLE","Technology":"XLK",
    "Communication Services":"XLC",
    "Consumer Staples":"XLP",
    "Health Care":"XLV",
    "Consumer Cyclical":"XLY",
    "Industrial":"XLI",
    "Utilities":"XLU",
    "Materials":"XLB",
    "Financial":"XLF",
    "Real Estate":"XLRE"}
    sectorDfList = []
    for sector in SectorDict.values():
        sectorDfList.append(pd.read_pickle("stockData/sectorCharts/"+sector+".pkl"))
        goodSectorDf = sectorDfList[0].drop(['open','high','low','adjclose','close','ticker','volume','percent','MA30','MA30Slope','RS'],axis=1)
        goodSectorDf['Sectors'] = [[] for _ in range(len(goodSectorDf))]
        goodSectorDf.index = sectorDfList[0].index
        index = 0
        for date in sectorDfList[0].index:
            listOfSector = []
            for df in sectorDfList:
                try:
                    # print(df.at[date,'MA30Slope'],df.at[date,'RS'])
                    # print(individual[0][9],individual[0][10])
                    if  df.at[date,'MA30Slope'] > individual[0][9] and df.at[date,'RS'] > individual[0][10]:
                        if df.at[date,'close'] > df.at[date,'MA30']:
                            # print(df.at[date,'ticker'])
                            listOfSector.append(df.at[date,'ticker'])
                except: 
                    pass
                # print(listOfSector)
            goodSectorDf.iat[index,0] = set(listOfSector)
            index = index + 1
    stockStageEstimator = StockStageEstimator(individual[0], goodSectorDf)
    scores = stockStageEstimator.score()
    debugFile = open("estimatorData/debug"+date.today().strftime('%Y-%m-%d')+".txt","a")
    debugFile.write(str(individual[0])+"\n")
    if scores == -1:
        debugFile.write(str(np.average(scores))+"\n")
        debugFile.write(str(stockStageEstimator.getReturns())+"\n")
        debugFile.close()
        return 0,
    result = stockStageEstimator.result()
    debugFile.write(str(np.average(scores[0]))+"\n")
    debugFile.write(str(np.average(scores[1]))+"\n")
    debugFile.write(str(stockStageEstimator.getReturns())+"\n")
    debugFile.write(str(result)+"\n")
    debugFile.close()
    if np.average(scores[0]) >  0.1:
        return 0,
    if np.average(scores[1]) > 1.3:
        return 0,
    paramFile = open("estimatorData/params"+date.today().strftime('%Y-%m-%d')+".txt","a")
    paramFile.write(str(individual[0])+"\n")
    paramFile.write(str(result)+"\n")
    paramFile.close()

In [ ]:
    # cur.execute("INSERT INTO Params (param,result) VALUES (?, ?)", (individual[0], result))

    resultFile = open("estimatorData/resultML"+date.today().strftime('%Y-%m-%d')+".txt","a")
    resultFile.write(str(individual[0])+"\n")
    resultFile.write(str(np.average(scores[0]))+"\n")
    resultFile.write(str(np.average(scores[1]))+"\n")
    resultFile.write(str(stockStageEstimator.getReturns())+"\n")
    resultFile.write(str(result)+"\n")
    resultFile.close()
    return result,


In [ ]:
### DEAP Evolutionary Functions ###

## Generating individuals
def generate_random_num_attr():
  original = np.array([0,  # 0 - Five Year High
                        0, # 1 - Volume Perc
                        0, # 2 - RS
                        0, # 3 - Slope
                        0, # 4 - Price v WMA30
                        0, # 5 - Price v Initial Support
                        0, # 6 - Prev Trough v Peak
                        0, # 7 - Price v Prev Support
                        0, # 8 - Price v Prev Close
                        0.995, # 9 - Sector MA30 Slope
                        0, # 10 - Sector RS
                        0.9, # 11 - SPDF Price v WMA30
                        0.995, # 12 - SPDF WMA30 Slope
                        0.85, # 13 - SPDF Sell P v WMA30
                        0.985]) # 14 - SPDF Sell WMA30 Slope
  noises = []
  sum_list = []

In [ ]:
  noises.append(random.uniform(0,1))
  noises.append(random.uniform(0.1,2.5))
  noises.append(random.uniform(-0.1,0.4))
  noises.append(random.uniform(0.9,1.1))
  noises.append(random.uniform(1,1.4))
  noises.append(random.uniform(1, 1.3))
  noises.append(random.uniform(0.8, 1))
  noises.append(random.uniform(0.8, 1.1))
  noises.append(random.uniform(1.2, 1.5))
  noises.append(np.random.normal(0, 0.008))
  noises.append(np.random.normal(0, 0.2))

  noises.append(np.random.normal(0, 0.05))
  noises.append(np.random.normal(0, 0.008))
  noises.append(np.random.normal(0, 0.05))
  noises.append(np.random.normal(0, 0.008))

  while original[11] + noises[11] < original[13] + noises[13]:
    noises[11] = np.random.normal(0, 0.05)
    noises[13] = np.random.normal(0, 0.05)
  while original[12] + noises[12] < original[14] + noises[14]:
    noises[12] = np.random.normal(0, 0.004)
    noises[14] = np.random.normal(0, 0.004)
  
  for (item1, item2) in zip(original, noises):
      sum_list.append(item1+item2)
  
  return sum_list

In [ ]:
## Mutating Individuals
def mutate(individual):
    size = len(individual)
    for i in range(0,size):
        if random.random() > 0.9:
            individual[i] = individual[i] + mutationNoises[i]
    return individual,